# Deconvolutional Neural Network

This notebook describes how the DCNN was built and trained.

single-cell or bulk-sorted RNA sequencing data can be used to learn molecular signatures of distinct cell types from a small collection of biospecimens. These signatures can then be repeatedly applied to characterize cellular heterogeneity

## Importing Stuff

In [2]:
import numpy as np
import h5py
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2DTranspose, Conv2D
import scanpy as sc

## File / Data Preprocessing

### Viewing UCD single cell test data.

In [4]:
# Load the .h5ad file
adata = sc.read_h5ad('pbmc3k_raw.h5ad')

# Print some basic information about the AnnData object
print(adata)

# View the first few rows of the data matrix
print("Viewing first five rows")
print(adata.X[:5])  # View the first 5 rows of the data matrix

# View the observation (cell) annotations
print("View the cell annotations")
print(adata.obs.head())

# View the variable (gene) annotations
print("View the gene annotations")
print(adata.var.head())



AnnData object with n_obs × n_vars = 2700 × 32738
    var: 'gene_ids'
Viewing first five rows
  (0, 70)	1.0
  (0, 166)	1.0
  (0, 178)	2.0
  (0, 326)	1.0
  (0, 363)	1.0
  (0, 410)	1.0
  (0, 412)	1.0
  (0, 492)	41.0
  (0, 494)	1.0
  (0, 495)	1.0
  (0, 496)	1.0
  (0, 525)	1.0
  (0, 556)	2.0
  (0, 558)	6.0
  (0, 671)	1.0
  (0, 684)	1.0
  (0, 735)	1.0
  (0, 770)	1.0
  (0, 793)	1.0
  (0, 820)	1.0
  (0, 859)	2.0
  (0, 871)	1.0
  (0, 908)	15.0
  (0, 926)	1.0
  (0, 941)	1.0
  :	:
  (4, 31257)	1.0
  (4, 31390)	1.0
  (4, 31488)	1.0
  (4, 31493)	1.0
  (4, 31661)	1.0
  (4, 31665)	1.0
  (4, 31714)	1.0
  (4, 31728)	1.0
  (4, 31759)	1.0
  (4, 31859)	1.0
  (4, 31923)	1.0
  (4, 31945)	2.0
  (4, 31963)	2.0
  (4, 31970)	1.0
  (4, 32008)	1.0
  (4, 32022)	3.0
  (4, 32436)	1.0
  (4, 32595)	1.0
  (4, 32611)	1.0
  (4, 32645)	2.0
  (4, 32696)	1.0
  (4, 32698)	7.0
  (4, 32702)	1.0
  (4, 32706)	2.0
  (4, 32708)	1.0
View the cell annotations
Empty DataFrame
Columns: []
Index: [AAACATACAACCAC-1, AAACATTGAGCTAC-1, A

### Viewing the PBMC data

Inspecting the datasets within the "matrix" group.

In [5]:
with h5py.File('Human_PBMC_TotalSeqB_3p_nextgem_gemx_nobatchcorrect_count_filtered_feature_bc_matrix.h5', 'r') as f:
    # Specify the group name
    group_name = 'matrix'

    # Access the group
    group = f[group_name]

    # Print group attributes
    print("Group name:", group_name)
    # As it's a group, it won't have a shape attribute
    # Add more attributes as needed

    # Print the datasets within the group
    print("Datasets within the group:")
    for dataset_name in group:
        print(dataset_name)

    # Access and inspect the datasets
    for dataset_name in group:
        dataset = group[dataset_name]
        if isinstance(dataset, h5py.Dataset):
            print("\nDataset:", dataset_name)
            print("Shape:", dataset.shape)
            print("dtype:", dataset.dtype)
            print("Preview of dataset values:")
            print(dataset[:15])  # Print the first five rows of the dataset


Group name: matrix
Datasets within the group:
barcodes
data
features
indices
indptr
shape

Dataset: barcodes
Shape: (15554,)
dtype: |S18
Preview of dataset values:
[b'AAACCCAAGCTTCGTA-1' b'AAACCCAAGGCACTCC-1' b'AAACCCACACGGCACT-1'
 b'AAACCCATCAACTGGT-1' b'AAACCCATCTACAGGT-1' b'AAACGAAGTCTGCCTT-1'
 b'AAACGAAGTCTTACTT-1' b'AAACGAAGTTAATGAG-1' b'AAACGAATCGACGAGA-1'
 b'AAACGCTCACTTCTCG-1' b'AAACGCTCATCCTAAG-1' b'AAACGCTCATGGATCT-1'
 b'AAACGCTGTCACTTCC-1' b'AAACGCTGTGCCCAGT-1' b'AAACGCTTCACTACGA-1']

Dataset: data
Shape: (58842498,)
dtype: int32
Preview of dataset values:
[ 5  1  1  1  3  2  1  3  1  4  1  1  1 13  3]

Dataset: indices
Shape: (58842498,)
dtype: int64
Preview of dataset values:
[ 36  51  60  64  67  69  81  88  93  94 121 152 156 180 200]

Dataset: indptr
Shape: (15555,)
dtype: int64
Preview of dataset values:
[    0  3672  6671  9762 12903 14496 17012 20677 23813 26149 29929 33118
 39367 42077 45649]

Dataset: shape
Shape: (2,)
dtype: int32
Preview of dataset values:
[38616

Inspecting the datasets within the "features" group.

In [14]:
# Open the HDF5 file
with h5py.File('Human_PBMC_TotalSeqB_3p_nextgem_gemx_nobatchcorrect_count_filtered_feature_bc_matrix.h5', 'r') as file:
    # Navigate to the parent group
    parent_group = file['matrix']
    
    # Navigate to the child group within the parent group
    child_group = parent_group['features']
    
    # Now you can work with datasets or subgroups within the child group
    # For example, to access a dataset within the child group:
    for dataset_name in child_group:
        dataset = child_group[dataset_name]
        print("\n" + dataset_name + "\n")
        print(dataset[:10])  # Print the first 10 elements of the dataset




_all_tag_keys

[b'genome' b'read' b'pattern' b'sequence']

feature_type

[b'Gene Expression' b'Gene Expression' b'Gene Expression'
 b'Gene Expression' b'Gene Expression' b'Gene Expression'
 b'Gene Expression' b'Gene Expression' b'Gene Expression'
 b'Gene Expression']

genome

[b'GRCh38' b'GRCh38' b'GRCh38' b'GRCh38' b'GRCh38' b'GRCh38' b'GRCh38'
 b'GRCh38' b'GRCh38' b'GRCh38']

id

[b'ENSG00000290825' b'ENSG00000243485' b'ENSG00000237613'
 b'ENSG00000290826' b'ENSG00000186092' b'ENSG00000238009'
 b'ENSG00000239945' b'ENSG00000239906' b'ENSG00000241860'
 b'ENSG00000241599']

name

[b'DDX11L2' b'MIR1302-2HG' b'FAM138A' b'ENSG00000290826' b'OR4F5'
 b'ENSG00000238009' b'ENSG00000239945' b'ENSG00000239906'
 b'ENSG00000241860' b'ENSG00000241599']

pattern

[b'' b'' b'' b'' b'' b'' b'' b'' b'' b'']

read

[b'' b'' b'' b'' b'' b'' b'' b'' b'' b'']

sequence

[b'' b'' b'' b'' b'' b'' b'' b'' b'' b'']


The dataset "data" is what contains the unique molecular identifier values, which are the principal expression quantification schemes used in scRNA-seq analysis.

### Looking through the Neuron dataset.

Inspecting the datasets within the "matrix" group.

In [3]:
# Open the HDF5 file
with h5py.File('10k_Mouse_Neurons_3p_nextgem_Multiplex_count_raw_feature_bc_matrix.h5', 'r') as f:
    # Specify the group name
    group_name = 'matrix'

    # Access the group
    group = f[group_name]

    # Print group attributes
    print("Group name:", group_name)
    # As it's a group, it won't have a shape attribute
    # Add more attributes as needed

    # Print the datasets within the group
    print("Datasets within the group:")
    for dataset_name in group:
        print(dataset_name)

    # Access and inspect the datasets
    for dataset_name in group:
        dataset = group[dataset_name]
        if isinstance(dataset, h5py.Dataset):
            print("\nDataset:", dataset_name)
            print("Shape:", dataset.shape)
            print("dtype:", dataset.dtype)
            print("Preview of dataset values:")
            print(dataset[:50])  # Print the first five rows of the dataset


Group name: matrix
Datasets within the group:
barcodes
data
features
indices
indptr
shape

Dataset: barcodes
Shape: (1408818,)
dtype: |S45
Preview of dataset values:
[b'AAACCCAAGAAACCAT-1' b'AAACCCAAGAAACTAC-1' b'AAACCCAAGAAACTCA-1'
 b'AAACCCAAGAAACTGT-1' b'AAACCCAAGAAATCCA-1' b'AAACCCAAGAAATTGC-1'
 b'AAACCCAAGAACACCA-1' b'AAACCCAAGAACCTCC-1' b'AAACCCAAGAACTGAT-1'
 b'AAACCCAAGAACTTCC-1' b'AAACCCAAGAAGATCT-1' b'AAACCCAAGAAGCGAA-1'
 b'AAACCCAAGAAGCGCT-1' b'AAACCCAAGAAGCTCG-1' b'AAACCCAAGAAGGTAG-1'
 b'AAACCCAAGAAGTCCG-1' b'AAACCCAAGAATAACC-1' b'AAACCCAAGAATCCCT-1'
 b'AAACCCAAGAATCTAG-1' b'AAACCCAAGAATTCAG-1' b'AAACCCAAGAATTTGG-1'
 b'AAACCCAAGACAACCA-1' b'AAACCCAAGACAACTA-1' b'AAACCCAAGACAAGCC-1'
 b'AAACCCAAGACACACG-1' b'AAACCCAAGACAGTCG-1' b'AAACCCAAGACATACA-1'
 b'AAACCCAAGACATATG-1' b'AAACCCAAGACATCAA-1' b'AAACCCAAGACATGCG-1'
 b'AAACCCAAGACCAAGC-1' b'AAACCCAAGACCACGA-1' b'AAACCCAAGACCATAA-1'
 b'AAACCCAAGACCATGG-1' b'AAACCCAAGACCATTC-1' b'AAACCCAAGACCCTCA-1'
 b'AAACCCAAGACCGCCT-1' b'AAACC

Looking through the datasets in the "features" group.

In [6]:
with h5py.File('10k_Mouse_Neurons_3p_nextgem_Multiplex_count_raw_feature_bc_matrix.h5', 'r') as f:
    # Specify the group name
    group_name = 'matrix'

    # Access the group
    group = f[group_name]

    # Check if "features" subgroup exists within the "matrix" group
    features_group = group['features']
        
    print("Attributes of 'features' subgroup:")
    for attr_name, attr_value in features_group.attrs.items():
        print(f"{attr_name}: {attr_value}")

    # You can also check if there are any datasets within the "features" subgroup
    print("\nDatasets within 'features' subgroup:")
    for dataset_name in features_group:
        dataset = features_group[dataset_name]
        print("\nDataset:", dataset_name)
        print("Shape:", dataset.shape)
        print("dtype:", dataset.dtype)
        print("Preview of dataset values:")
        print(dataset[:10])  # Print the first five rows of the dataset

Attributes of 'features' subgroup:

Datasets within 'features' subgroup:

Dataset: _all_tag_keys
Shape: (1,)
dtype: |S256
Preview of dataset values:
[b'genome']

Dataset: feature_type
Shape: (33696,)
dtype: |S256
Preview of dataset values:
[b'Gene Expression' b'Gene Expression' b'Gene Expression'
 b'Gene Expression' b'Gene Expression' b'Gene Expression'
 b'Gene Expression' b'Gene Expression' b'Gene Expression'
 b'Gene Expression']

Dataset: genome
Shape: (33696,)
dtype: |S256
Preview of dataset values:
[b'GRCm39' b'GRCm39' b'GRCm39' b'GRCm39' b'GRCm39' b'GRCm39' b'GRCm39'
 b'GRCm39' b'GRCm39' b'GRCm39']

Dataset: id
Shape: (33696,)
dtype: |S256
Preview of dataset values:
[b'ENSMUSG00000051951' b'ENSMUSG00000089699' b'ENSMUSG00000102331'
 b'ENSMUSG00000102343' b'ENSMUSG00000025900' b'ENSMUSG00000025902'
 b'ENSMUSG00000104238' b'ENSMUSG00000104328' b'ENSMUSG00000033845'
 b'ENSMUSG00000120403']

Dataset: name
Shape: (33696,)
dtype: |S256
Preview of dataset values:
[b'Xkr4' b'Gm1992' b'Gm1

Other ways of data preprocessing.

In [25]:
# Step 1: Data Preprocessing
def load_data(file_path):
    with h5py.File(file_path, 'r') as f:
        # Inspect keys
        print(list(f.keys()))
        # Load the correct dataset
        data = np.array(f['matrix'])
    # Perform any necessary preprocessing such as normalization or scaling
    return data


For supervised learning, create the labels and training and testing dataset.

In [ ]:
# Another way of loading in the data and creating test / train variables.
# Load MNIST data
f = h5py.File('./train.hdf5', 'r')
input_train = f['image'][...]
label_train = f['label'][...]
f.close()
f = h5py.File('./test.hdf5', 'r')
input_test = f['image'][...]
label_test = f['label'][...]
f.close()

# Reshape data
input_train = input_train.reshape((len(input_train), img_width, img_height, img_num_channels))
input_test  = input_test.reshape((len(input_test), img_width, img_height, img_num_channels))

In [26]:
pip install cellranger

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement cellranger (from versions: none)
ERROR: No matching distribution found for cellranger


In [13]:
import cellranger.matrix as cr_matrix
filtered_h5 = 'Human_PBMC_TotalSeqB_3p_nextgem_gemx_nobatchcorrect_count_filtered_feature_bc_matrix.h5'
filtered_matrix_h5 = cr_matrix.CountMatrix.load_h5_file(filtered_h5)


import collections
import scipy.sparse as sp_sparse
import tables

CountMatrix = collections.namedtuple('CountMatrix', ['feature_ref', 'barcodes', 'matrix'])

def get_matrix_from_h5(filename):
    with tables.open_file(filename, 'r') as f:
        mat_group = f.get_node(f.root, 'matrix')
        barcodes = f.get_node(mat_group, 'barcodes').read()
        data = getattr(mat_group, 'data').read()
        indices = getattr(mat_group, 'indices').read()
        indptr = getattr(mat_group, 'indptr').read()
        shape = getattr(mat_group, 'shape').read()
        matrix = sp_sparse.csc_matrix((data, indices, indptr), shape=shape)

        feature_ref = {}
        feature_group = f.get_node(mat_group, 'features')
        feature_ids = getattr(feature_group, 'id').read()
        feature_names = getattr(feature_group, 'name').read()
        feature_types = getattr(feature_group, 'feature_type').read()
        feature_ref['id'] = feature_ids
        feature_ref['name'] = feature_names
        feature_ref['feature_type'] = feature_types
        tag_keys = getattr(feature_group, '_all_tag_keys').read()
        for key in tag_keys:
            feature_ref[key] = getattr(feature_group, key.decode()).read()

        return CountMatrix(feature_ref, barcodes, matrix)



filtered_h5 = "/opt/sample345/outs/filtered_feature_bc_matrix.h5"
filtered_matrix_h5 = get_matrix_from_h5(filtered_h5)


ModuleNotFoundError: No module named 'cellranger'

### Viewing the GEO files

In [7]:
import scipy.io

# Load the .mtx file
matrix = scipy.io.mmread('matrix.mtx')

# Print the matrix
print(matrix)


  (13, 0)	2
  (20, 0)	2
  (30, 0)	1
  (34, 0)	1
  (40, 0)	1
  (42, 0)	5
  (50, 0)	3
  (52, 0)	2
  (71, 0)	1
  (123, 0)	1
  (131, 0)	2
  (149, 0)	1
  (175, 0)	1
  (178, 0)	1
  (182, 0)	2
  (189, 0)	1
  (198, 0)	3
  (199, 0)	1
  (206, 0)	1
  (223, 0)	3
  (233, 0)	2
  (234, 0)	1
  (244, 0)	1
  (245, 0)	2
  (248, 0)	5
  :	:
  (31341, 4431)	6
  (31343, 4431)	1
  (31345, 4431)	1
  (31378, 4431)	2
  (31379, 4431)	1
  (31392, 4431)	18
  (31393, 4431)	3
  (31395, 4431)	4
  (31414, 4431)	3
  (31415, 4431)	2
  (31422, 4431)	1
  (31450, 4431)	19
  (31463, 4431)	7
  (32193, 4431)	12
  (32195, 4431)	18
  (32196, 4431)	19
  (32197, 4431)	102
  (32198, 4431)	56
  (32200, 4431)	61
  (32201, 4431)	74
  (32202, 4431)	3
  (32203, 4431)	1
  (32204, 4431)	17
  (32205, 4431)	1
  (32207, 4431)	36


## Creating the Input Matrices

Deconvolution can be modeled by E = S * C, where:

- E is the matrix of bulk tissue level feature representation
- S is the matrix of cell-type specific features (signature / reference matrix)
- C is the cell-type proportion matrix

### Creating Signature Matrix, S

S = feature IDs (genes) * cell type

It is a specialized expression matrix of cell type-specific "barcode" genes, often called a "signature matrix".

Each row is a gene, each column is a single-cell transcriptome.

### Creating Cell-Type Proportion Matrix, C

C = 

### Creating Feature Representation Matrix, E

## Building the Model's Architecture

Model Configuration (put towards the beginning later on)

In [ ]:
# Model configuration
batch_size = 50
img_width, img_height, img_num_channels = 28, 28, 1
loss_function = sparse_categorical_crossentropy
no_classes = 10
no_epochs = 25
optimizer = Adam()
validation_split = 0.2
verbosity = 1

This model uses a max pooling layer. It is a down-sampling operation that reducte the spatial dimensions of the feature maps. Works by partitioning the input feature map into non overlapping rectangles and for each subregion outputs the max value. Used in the encoder portion to reduce spatial dimension but increase depth.

The Upsampling Layer, increases the spatial dimensions of the feature maps. It is used to recover the spatial information lost during the downsampling, and produce hihger resolution feature maps (some upsampling techniques include NN-interpolation, bilinear interpolation, transposed convolution). SIMPLE AND COMPUTATIONALLY EFFICIENT.

In [30]:

# Step 2: Model Architecture
def build_autoencoder(input_shape):
    model = models.Sequential([
        layers.Input(shape=input_shape),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2), padding='same'),
        layers.Conv2D(16, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2), padding='same'),
        layers.Conv2D(8, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2), padding='same'),

        
        layers.Conv2D(8, (3, 3), activation='relu', padding='same'),
        layers.UpSampling2D((2, 2)),
        layers.Conv2D(16, (3, 3), activation='relu', padding='same'),
        layers.UpSampling2D((2, 2)),
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.UpSampling2D((2, 2)),
        layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model



Hidden layers are the convolution and pooling.
In a convolution we take a filter of a small dimension and move it across an image.
The filter's values are tuned iteratively during training.
Pooling layers help reduce the amount of parameters, reduce computation.
Max pooling selects the maximum value within that pool.


The transpose convolution (deconvolution), does the inverse of the convolution operation. It is used to increase the spatial dimensions of feature maps and can be thought of as learning to fill in the missing spatial information. ABLE TO LEARN A SET OF TRAINABLE PARAMETERS.

In [ ]:
# Define the deconvolutional neural network architecture
def create_deconv_nn(input_shape):
    model = models.Sequential([
        # Encoder
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        layers.MaxPooling2D((2, 2), padding='same'),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2), padding='same'),
        
        # Decoder
        layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same'),
        layers.UpSampling2D((2, 2)),
        layers.Conv2DTranspose(32, (3, 3), activation='relu', padding='same'),
        layers.UpSampling2D((2, 2)),
        layers.Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')
    ])
    return model


# Create the deconvolutional neural network model
input_shape = (28, 28, 1)
model = create_deconv_nn(input_shape)

## Compiling the Model

In [ ]:
# Step 3: Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy')

## Training the Model

In [ ]:
# Train the model
model.fit(train_images, train_images, epochs=10, batch_size=128, validation_split=0.1)


In [ ]:
# Step 4: Model Building and Training
input_shape = single_cell_data.shape[1:]
autoencoder = build_autoencoder(input_shape)
autoencoder.fit(single_cell_data, single_cell_data, epochs=10, batch_size=32, shuffle=True)

# Step 5: Model Evaluation (Optional)
# Evaluate the trained model using appropriate metrics

# Step 6: Deployment (Optional)
# Save the trained model for future use
#autoencoder.save('single_cell_autoencoder_model.h5')

In [ ]:
# Train the model
model.fit(train_images, train_images, epochs=10, batch_size=128, validation_split=0.1)


## Evaluating the Model

In [18]:
# Evaluate the model
test_loss = model.evaluate(test_images, test_images)
print('Test loss:', test_loss)

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.0640
Test loss: 0.06468354910612106
